In [2]:
import numpy as np
import pandas as pd
from myfunctions import BPstar
from myfunctions import BPstarmodel
import matplotlib.pyplot as plt
import os
from est_model import estimate_model

In [3]:
parameters = {
    'M1': {
        'A': np.array([[0.3, 0.2], [0.1, 0.3]]),
        'B': np.array([[0.1, 0.2], [0.3, 0.2]]),
        'omeg': np.array([0.5, 0.3]),
        'delt': 0.4
        
    },
    'M2': {
        'A': np.array([[0.2, 0], [0, 0.4]]),
        'B': np.array([[0.5, 0], [0, 0.3]]),
        'omeg': np.array([0.3, 0.5]),
        'delt': 0.7
    },
    'M3': {
        'A': np.array([[0.2, 0.0], [0.0, 0.4]]),
        'B': np.array([[0.5, 0.1], [0.3, 0.2]]),
        'omeg': np.array([0.3, 0.5]),
        'delt': 0.7
    },
    'M4': {
        'A': np.array([[0.2, 0.3], [0.4, 0.2]]),
        'B': np.array([[0.2, 0], [0, 0.3]]),
        'omeg': np.array([0.5, 0.3]),
        'delt': -0.5
    }
}

In [4]:
for model_name in parameters:
    parameters[model_name]['real_params'] = [
        parameters[model_name]['omeg'][0],
        *parameters[model_name]['A'][0],
        *parameters[model_name]['B'][0],
        parameters[model_name]['omeg'][1],
        *parameters[model_name]['A'][1],
        *parameters[model_name]['B'][1],
        parameters[model_name]['delt']
    ] 
    print(parameters[model_name]['real_params'])

[0.5, 0.3, 0.2, 0.1, 0.2, 0.3, 0.1, 0.3, 0.3, 0.2, 0.4]
[0.3, 0.2, 0.0, 0.5, 0.0, 0.5, 0.0, 0.4, 0.0, 0.3, 0.7]
[0.3, 0.2, 0.0, 0.5, 0.1, 0.5, 0.0, 0.4, 0.3, 0.2, 0.7]
[0.5, 0.2, 0.3, 0.2, 0.0, 0.3, 0.4, 0.2, 0.0, 0.3, -0.5]


In [37]:
#to save the full vector of params with delta to a file:
model_specs = {
    'M1': {'indices': range(11)},
    'M2': {'indices': [0, 1, 3, 5, 7, 9, 10]},
    'M3': {'indices': [0, 1, 3, 4, 5, 7, 8, 9, 10]},
    'M4': {'indices': [0, 1, 2, 3, 5, 6, 7, 9, 10]}
}

base_folder = "estimation"
os.makedirs(base_folder, exist_ok=True)
sub_folder = os.path.join(base_folder,"parameters")
os.makedirs(sub_folder, exist_ok=True)
for model in parameters:
    real_values = np.array([parameters[model]['real_params'][i] for i in model_specs[model]['indices']])
    print(real_values)
    filename = f"real_parameters_{model}.csv"
    filepath = os.path.join(sub_folder, filename)
    np.savetxt(filepath, real_values, delimiter=",")
    print(f"saved real parameter values for model {model} to {filepath}")
    


[0.5 0.3 0.2 0.1 0.2 0.3 0.1 0.3 0.3 0.2 0.4]
saved real parameter values for model M1 to estimation\parameters\real_parameters_M1.csv
[0.3 0.2 0.5 0.5 0.4 0.3 0.7]
saved real parameter values for model M2 to estimation\parameters\real_parameters_M2.csv
[0.3 0.2 0.5 0.1 0.5 0.4 0.3 0.2 0.7]
saved real parameter values for model M3 to estimation\parameters\real_parameters_M3.csv
[ 0.5  0.2  0.3  0.2  0.3  0.4  0.2  0.3 -0.5]
saved real parameter values for model M4 to estimation\parameters\real_parameters_M4.csv


In [5]:
base_folder = "estimation"  # Or wherever your data is stored
sizes = [200, 500, 1000]

In [39]:
# Process and save estimation data
for model in parameters:
    for size in sizes:
        model_folder = os.path.join(base_folder, model)
        filename = f"theta_hat_{model}_{size}.csv"
        filepath = os.path.join(model_folder, filename)

        repestim = np.loadtxt(filepath, delimiter=',')
        print(f"Loaded data: {filepath} with shape {np.shape(repestim)}")

        current_estim = np.array([repestim[i, :] for i in model_specs[model]['indices']])
        current_realpar = np.array([parameters[model]['real_params'][i] for i in model_specs[model]['indices']])
        meanest = current_estim.mean(axis=1)
        stdest = current_estim.std(axis=1)
        minest = current_estim.min(axis=1)
        maxest = current_estim.max(axis=1)
        medianest = np.median(current_estim, axis=1)

        size_folder = os.path.join(model_folder, f"size_{size}")
        os.makedirs(size_folder, exist_ok=True)
        festimname = f"modelestim_{model}_{size}.csv"
        festimpath = os.path.join(size_folder, festimname)

        np.savetxt(festimpath, current_estim, delimiter=",")
        print(f"Saved final size estimation to: {festimname} with shape {np.shape(current_estim)}")

        # Create DataFrame
        estimstat = pd.DataFrame({
            'Parameter': [f'Parameter {i + 1}' for i in range(current_estim.shape[0])],
            'Mean': meanest,
            'Median': medianest,
            'Std': stdest,
            'Min': minest,
            'Max': maxest,
            'Real Value': current_realpar
        })

        # Save DataFrame
        estimstat_filename = f"estimstat_{model}_{size}.csv"
        estimstat_filepath = os.path.join(size_folder, estimstat_filename)
        estimstat.to_csv(estimstat_filepath, index=False)
        print(f"Saved statistics DataFrame to: {estimstat_filename}")

Loaded data: estimation\M1\theta_hat_M1_200.csv with shape (199, 11)
Saved final size estimation to: modelestim_M1_200.csv with shape (11, 11)
Saved statistics DataFrame to: estimstat_M1_200.csv
Loaded data: estimation\M1\theta_hat_M1_500.csv with shape (199, 11)
Saved final size estimation to: modelestim_M1_500.csv with shape (11, 11)
Saved statistics DataFrame to: estimstat_M1_500.csv
Loaded data: estimation\M1\theta_hat_M1_1000.csv with shape (199, 11)
Saved final size estimation to: modelestim_M1_1000.csv with shape (11, 11)
Saved statistics DataFrame to: estimstat_M1_1000.csv
Loaded data: estimation\M2\theta_hat_M2_200.csv with shape (199, 7)
Saved final size estimation to: modelestim_M2_200.csv with shape (7, 7)
Saved statistics DataFrame to: estimstat_M2_200.csv
Loaded data: estimation\M2\theta_hat_M2_500.csv with shape (199, 7)
Saved final size estimation to: modelestim_M2_500.csv with shape (7, 7)
Saved statistics DataFrame to: estimstat_M2_500.csv
Loaded data: estimation\M2\t

In [40]:
# Combine data across sizes
for model in parameters:
    all_data = []  # List to store DataFrames for different sizes

    for size in sizes:
        # Load existing DataFrame for the current size
        size_folder = os.path.join(base_folder, model, f"size_{size}")
        estimstat_filename = f"estimstat_{model}_{size}.csv"
        estimstat_filepath = os.path.join(size_folder, estimstat_filename)
        estimstat_df = pd.read_csv(estimstat_filepath)

        # Add a 'Size' column to the DataFrame
        estimstat_df['Size'] = size

        # Append the DataFrame to the list
        all_data.append(estimstat_df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    combined_filename = f"combined_data_{model}.csv"
    combined_filepath = os.path.join(base_folder, model, combined_filename)
    combined_df.to_csv(combined_filepath, index=False)

    print(f"Combined data for model {model} saved to: {combined_filepath}")

Combined data for model M1 saved to: estimation\M1\combined_data_M1.csv
Combined data for model M2 saved to: estimation\M2\combined_data_M2.csv
Combined data for model M3 saved to: estimation\M3\combined_data_M3.csv
Combined data for model M4 saved to: estimation\M4\combined_data_M4.csv


In [16]:
# Initialize the errors dictionary
errors = {model: {'MADE': [], 'MSE': []} for model in parameters}

# Define the number of repetitions (assuming nrep is defined somewhere in your context)
nrep = 200  # Replace with the actual number of repetitions

# Process the errors for each model and size
for model in parameters:
    model_folder = os.path.join(base_folder, model)
    for size in sizes:
        size_folder = os.path.join(model_folder, f"size_{size}")
        filename = f"modelestim_{model}_{size}.csv"
        filepath = os.path.join(size_folder, filename)

        data = np.loadtxt(filepath, delimiter=',')
        num_cols = data.shape[1]
        print(f"Loaded data: {filename} with shape {np.shape(data)}")

        # Load the statistics DataFrame
        estimstat_filename = f"estimstat_{model}_{size}.csv"
        estimstat_filepath = os.path.join(size_folder, estimstat_filename)
        estimstat_df = pd.read_csv(estimstat_filepath)

        # Access the 'Real Value' column as a NumPy array
        real_par = estimstat_df['Real Value'].values
        print(f"Real parameters: {real_par}")

        # Initialize MADE and MSE
        MADE = np.zeros(len(real_par))
        MSE = np.zeros(len(real_par))

        # Ensure nrep does not exceed the number of columns in data
        nrep = min(nrep, num_cols)
        print(f"Using {nrep} repetitions for calculations")

        # Calculate MADE and MSE for each repetition
        for i in range(nrep):
            MADE += abs(data[:, i] - real_par) / nrep
            MSE += ((data[:, i] - real_par) ** 2) / nrep

        print(f"MADE = {MADE}")
        print(f"MSE = {MSE}")

        # Add to errors dictionary
        errors[model]['MADE'].append(MADE)
        errors[model]['MSE'].append(MSE)

        # Save MADE error
        errorname = f"MADE_{model}_{size}.csv"
        errorpath = os.path.join(size_folder, errorname)
        np.savetxt(errorpath, MADE, delimiter=',')
        print(f"Saved MADE error for model {model} with size {size} to {errorpath}")

        # Save MSE error
        errorname = f"MSE_{model}_{size}.csv"
        errorpath = os.path.join(size_folder, errorname)
        np.savetxt(errorpath, MSE, delimiter=',')
        print(f"Saved MSE error for model {model} with size {size} to {errorpath}")

Loaded data: modelestim_M1_200.csv with shape (11, 11)
Real parameters: [0.5 0.3 0.2 0.1 0.2 0.3 0.1 0.3 0.3 0.2 0.4]
Using 11 repetitions for calculations
MADE = [0.29572636 0.17499409 0.14620986 0.23662671 0.15189364 0.16013664
 0.27585726 0.20444736 0.18475786 0.18571725 0.22359782]
MSE = [0.10067788 0.05093714 0.03220793 0.08820452 0.05157159 0.05259936
 0.14656013 0.05363171 0.06440654 0.05920889 0.06914141]
Saved MADE error for model M1 with size 200 to estimation\M1\size_200\MADE_M1_200.csv
Saved MSE error for model M1 with size 200 to estimation\M1\size_200\MSE_M1_200.csv
Loaded data: modelestim_M1_500.csv with shape (11, 11)
Real parameters: [0.5 0.3 0.2 0.1 0.2 0.3 0.1 0.3 0.3 0.2 0.4]
Using 11 repetitions for calculations
MADE = [0.27747744 0.16421855 0.12513208 0.21471291 0.09866536 0.11182855
 0.23000969 0.15738191 0.13412291 0.18909509 0.199321  ]
MSE = [0.09681558 0.04396041 0.02372087 0.09708316 0.01565274 0.0219626
 0.10609102 0.04263724 0.02626465 0.07471089 0.0531288

In [17]:
# Define the folder structure
estimation_folder = "estimation"
models = ["M1", "M2", "M3", "M4"]  # Add all your models here
sizes = [200, 500, 1000]

# Create the figures/errors directory if it doesn't exist
output_dir = os.path.join("figures","MV", "errors")
os.makedirs(output_dir, exist_ok=True)

In [22]:
errors

{'M1': {'MADE': [array([0.29572636, 0.17499409, 0.14620986, 0.23662671, 0.15189364,
          0.16013664, 0.27585726, 0.20444736, 0.18475786, 0.18571725,
          0.22359782]),
   array([0.27747744, 0.16421855, 0.12513208, 0.21471291, 0.09866536,
          0.11182855, 0.23000969, 0.15738191, 0.13412291, 0.18909509,
          0.199321  ]),
   array([0.26757595, 0.11716145, 0.12542918, 0.20168127, 0.185058  ,
          0.16860134, 0.19160091, 0.09932818, 0.15688473, 0.12774422,
          0.18319025])],
  'MSE': [array([0.10067788, 0.05093714, 0.03220793, 0.08820452, 0.05157159,
          0.05259936, 0.14656013, 0.05363171, 0.06440654, 0.05920889,
          0.06914141]),
   array([0.09681558, 0.04396041, 0.02372087, 0.09708316, 0.01565274,
          0.0219626 , 0.10609102, 0.04263724, 0.02626465, 0.07471089,
          0.05312887]),
   array([0.08903603, 0.0158144 , 0.02666661, 0.06890357, 0.07675645,
          0.03697015, 0.07553759, 0.01387856, 0.03718952, 0.02403149,
          0.051103

In [21]:
# Plotting
for model in parameters:
    num_params = len(errors[model]['MADE'][0])
    fig, axes = plt.subplots(num_params, 1, figsize=(4.25, 2 * num_params), sharex=True)
    fig.suptitle(f"MADE and MSE for Model {model}", fontsize=10)

    bar_width = 0.3 

    handles = []
    labels = []

    for p in range(num_params):
        ax = axes[p]
        # Use 'i' to index into errors[model]['MADE'] and errors[model]['MSE']
        made_values = [errors[model]['MADE'][i][p] for i in range(len(sizes))] 
        mse_values = [errors[model]['MSE'][i][p] for i in range(len(sizes))]
        
        bar1 = ax.bar(np.arange(len(sizes)), made_values, bar_width, label='MADE')
        bar2 = ax.bar(np.arange(len(sizes)) + bar_width, mse_values, bar_width, label='MSE')

        ax.set_xticks(np.arange(len(sizes)) + bar_width / 2)

        # Only set x-axis label for the bottom subplot
        if p == num_params - 1:
            ax.set_xticklabels(sizes, fontsize=8)
            ax.set_xlabel("Sample Size", fontsize=8) 
        else:
            ax.set_xticklabels([]) # Remove x-tick labels for other subplots

        ax.set_ylabel(f"Error (Parameter {p+1})", fontsize=8)

        # Remove spines
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)

        # Add horizontal dotted line separator (optional)
        if p < num_params - 1:
            ax.axhline(ax.get_ylim()[1], color='gray', linestyle='dotted', linewidth=0.8)

        # Collect handles and labels for the legend (only once)
        if p == 0:
            handles.append(bar1)
            handles.append(bar2)
            labels.append('MADE')
            labels.append('MSE')

    # Place the legend outside the plot
    fig.legend(handles, labels, loc='upper right', bbox_to_anchor=(1, 0.95), fontsize=8)

    # Save the figure
    plt.tight_layout()
    plt.subplots_adjust(top=0.90, bottom=0.1, hspace=0.3, right=0.8)
    outputname = f"errors_model_{model}.png" 
    fig.savefig(os.path.join(output_dir, outputname))
    print(f"saved figure to {output_dir} {outputname}")
    plt.close(fig) # Close the figure to free up memory

saved figure to figures\MV\errors errors_model_M1.png
saved figure to figures\MV\errors errors_model_M2.png
saved figure to figures\MV\errors errors_model_M3.png
saved figure to figures\MV\errors errors_model_M4.png
